In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import glob
import csv
import os
import sys

from datetime import datetime, timedelta
from tsmoothie.smoother import *
from tsmoothie.utils_func import sim_seasonal_data
from tsmoothie.smoother import DecomposeSmoother

In [2]:
#today = datetime.now()
today = datetime(2020, 11, 30)
last_date=today-timedelta(weeks=1)
today = today.strftime('%Y-%m-%d')#ex)2019-03-14
last_date = last_date.strftime('%Y-%m-%d')#ex)2019-03-14

In [73]:
import pymysql

conn = pymysql.connect(
    user='egov39exbd', 
    passwd='579nof83', 
    host='172.17.17.165', 
    db='egov39exbd', 
    charset='utf8'
)
cur = conn.cursor(pymysql.cursors.DictCursor)

selectQry = """\
    SELECT
        cstmr_no,
        mrnr_no,
        smrt_trmnl_no,
        mtinsp_dt,
        smrt_mtinsp_hour_usgqty
    FROM asmartmiteringinfo
    WHERE creat_stle = 'N'
    AND cstmr_no = '1970000026'
    AND mrnr_no = 'A00'
    AND mtinsp_dt BETWEEN '{last_date} 01:00:00' AND '{today} 00:00:00'
    AND SUBSTR(cstmr_no, 0,4) != '2007'
    ORDER BY cstmr_no, smrt_trmnl_no, mtinsp_dt
""".format(last_date=last_date, today=today)

print(selectQry)

cur.execute(selectQry)
rows = cur.fetchall()

df = pd.DataFrame(rows)
df.columns=['cstmr_no', 'mrnr_no', 'smrt_trmnl_no', 'mtinsp_dt', 'smrt_mtinsp_hour_usgqty']

    SELECT
        cstmr_no,
        mrnr_no,
        smrt_trmnl_no,
        mtinsp_dt,
        smrt_mtinsp_hour_usgqty
    FROM asmartmiteringinfo
    WHERE creat_stle = 'N'
    AND cstmr_no = '1970000026'
    AND mrnr_no = 'A00'
    AND mtinsp_dt BETWEEN '2020-11-23 01:00:00' AND '2020-11-30 00:00:00'
    AND SUBSTR(cstmr_no, 0,4) != '2007'
    ORDER BY cstmr_no, smrt_trmnl_no, mtinsp_dt



## 데이터 확인하기 (Viewing Data)

In [74]:
df.head() #첯행부터 5행까지

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
0,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
1,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000
3,1970000026,A00,1911WS0059893,2020-11-23 04:00:00,0.000
4,1970000026,A00,1911WS0059893,2020-11-23 05:00:00,0.000


In [75]:
df.tail() #끝행부터 5행까지

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
152,1970000026,A00,1911WS0059893,2020-11-29 20:00:00,0.000
153,1970000026,A00,1911WS0059893,2020-11-29 21:00:00,0.000
154,1970000026,A00,1911WS0059893,2020-11-29 22:00:00,0.000
155,1970000026,A00,1911WS0059893,2020-11-29 23:00:00,0.000
156,1970000026,A00,1911WS0059893,2020-11-30 00:00:00,0.000


In [78]:
df.index

RangeIndex(start=0, stop=157, step=1)

In [79]:
df.columns

Index(['cstmr_no', 'mrnr_no', 'smrt_trmnl_no', 'mtinsp_dt',
       'smrt_mtinsp_hour_usgqty'],
      dtype='object')

In [81]:
df_val = df.values

## 정렬

In [9]:
df.T # 열과 행을 바꾼 형태의 데이터프레임.

,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
cstmr_no,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,...,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026,1970000026
mrnr_no,A00,A00,A00,A00,A00,A00,A00,A00,A00,A00,...,A00,A00,A00,A00,A00,A00,A00,A00,A00,A00
smrt_trmnl_no,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,...,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893,1911WS0059893
mtinsp_dt,2020-11-23 01:00:00,2020-11-23 02:00:00,2020-11-23 03:00:00,2020-11-23 04:00:00,2020-11-23 05:00:00,2020-11-23 06:00:00,2020-11-23 07:00:00,2020-11-23 08:00:00,2020-11-23 09:00:00,2020-11-23 10:00:00,...,2020-11-29 14:00:00,2020-11-29 15:00:00,2020-11-29 16:00:00,2020-11-29 17:00:00,2020-11-29 18:00:00,2020-11-29 20:00:00,2020-11-29 21:00:00,2020-11-29 22:00:00,2020-11-29 23:00:00,2020-11-30 00:00:00
smrt_mtinsp_hour_usgqty,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.006,0.003,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [10]:
df.sort_index(axis=1, ascending=False) #칼럼 이름순으로 열정렬 (역순)

,smrt_trmnl_no,smrt_mtinsp_hour_usgqty,mtinsp_dt,mrnr_no,cstmr_no
0,1911WS0059893,0.000,2020-11-23 01:00:00,A00,1970000026
1,1911WS0059893,0.000,2020-11-23 02:00:00,A00,1970000026
2,1911WS0059893,0.000,2020-11-23 03:00:00,A00,1970000026
3,1911WS0059893,0.000,2020-11-23 04:00:00,A00,1970000026
4,1911WS0059893,0.000,2020-11-23 05:00:00,A00,1970000026
...,...,...,...,...,...
152,1911WS0059893,0.000,2020-11-29 20:00:00,A00,1970000026
153,1911WS0059893,0.000,2020-11-29 21:00:00,A00,1970000026
154,1911WS0059893,0.000,2020-11-29 22:00:00,A00,1970000026
155,1911WS0059893,0.000,2020-11-29 23:00:00,A00,1970000026


In [11]:
df.sort_values(by='cstmr_no') #값으로 정렬

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
0,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
100,1970000026,A00,1911WS0059893,2020-11-27 15:00:00,0.015
101,1970000026,A00,1911WS0059893,2020-11-27 16:00:00,0.000
102,1970000026,A00,1911WS0059893,2020-11-27 17:00:00,0.025
103,1970000026,A00,1911WS0059893,2020-11-27 18:00:00,0.002
...,...,...,...,...,...
53,1970000026,A00,1911WS0059893,2020-11-25 08:00:00,0.001
54,1970000026,A00,1911WS0059893,2020-11-25 09:00:00,0.000
55,1970000026,A00,1911WS0059893,2020-11-25 10:00:00,0.000
48,1970000026,A00,1911WS0059893,2020-11-25 02:00:00,0.000


## 데이터 선택

In [12]:
df['cstmr_no']

0      1970000026
1      1970000026
2      1970000026
3      1970000026
4      1970000026
          ...    
152    1970000026
153    1970000026
154    1970000026
155    1970000026
156    1970000026
Name: cstmr_no, Length: 157, dtype: object

In [13]:
df[0:3] #0번째부터 3번째미만 총 3개 가져옴

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
0,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
1,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000


In [14]:
len(df)

157

In [15]:
dates = pd.date_range('20210101', periods=len(df))

In [16]:
df.index=dates #인덱스에 숫자대신 날짜로 셋팅

In [17]:
df[0:3]

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
2021-01-01,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
2021-01-02,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2021-01-03,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000


In [18]:
df['20210101':'20210104']

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
2021-01-01,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
2021-01-02,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2021-01-03,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000
2021-01-04,1970000026,A00,1911WS0059893,2020-11-23 04:00:00,0.000


In [19]:
df.loc['20210606'] #인덱스 이름으로 행가지고 오기

cstmr_no                            1970000026
mrnr_no                                    A00
smrt_trmnl_no                    1911WS0059893
mtinsp_dt                  2020-11-30 00:00:00
smrt_mtinsp_hour_usgqty                  0.000
Name: 2021-06-06 00:00:00, dtype: object

In [20]:
df.loc[:,['cstmr_no', 'mtinsp_dt']] #특정 2칼럼의 전체 데이터 가져오기

,cstmr_no,mtinsp_dt
2021-01-01,1970000026,2020-11-23 01:00:00
2021-01-02,1970000026,2020-11-23 02:00:00
2021-01-03,1970000026,2020-11-23 03:00:00
2021-01-04,1970000026,2020-11-23 04:00:00
2021-01-05,1970000026,2020-11-23 05:00:00
...,...,...
2021-06-02,1970000026,2020-11-29 20:00:00
2021-06-03,1970000026,2020-11-29 21:00:00
2021-06-04,1970000026,2020-11-29 22:00:00
2021-06-05,1970000026,2020-11-29 23:00:00


In [21]:
len(df[df.smrt_mtinsp_hour_usgqty > 0])

49

In [82]:
df2 = df.copy()
df2

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
0,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
1,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000
3,1970000026,A00,1911WS0059893,2020-11-23 04:00:00,0.000
4,1970000026,A00,1911WS0059893,2020-11-23 05:00:00,0.000
...,...,...,...,...,...
152,1970000026,A00,1911WS0059893,2020-11-29 20:00:00,0.000
153,1970000026,A00,1911WS0059893,2020-11-29 21:00:00,0.000
154,1970000026,A00,1911WS0059893,2020-11-29 22:00:00,0.000
155,1970000026,A00,1911WS0059893,2020-11-29 23:00:00,0.000


### 조건식으로 데이터 선택

In [68]:
cond1 = df2['cstmr_no'] == '1970000026'
cond2 = df2['smrt_mtinsp_hour_usgqty'] > 0
dft = df2[cond1 & cond2]

In [69]:
dft = df2[(df2.cstmr_no == '1970000026') & (df2.smrt_mtinsp_hour_usgqty > 0)]

In [25]:
df2.dtypes

cstmr_no                           object
mrnr_no                            object
smrt_trmnl_no                      object
mtinsp_dt                  datetime64[ns]
smrt_mtinsp_hour_usgqty            object
dtype: object

In [26]:
df2[df2['mtinsp_dt'].isin(['2020-11-28 10:00:00', '2020-11-28 11:00:00', '2020-11-28 12:00:00'])]

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
2021-04-30,1970000026,A00,1911WS0059893,2020-11-28 10:00:00,0.015
2021-05-01,1970000026,A00,1911WS0059893,2020-11-28 11:00:00,0.000
2021-05-02,1970000026,A00,1911WS0059893,2020-11-28 12:00:00,0.000


## 결측치

In [66]:
df.fillna(value=0)

,cstmr_no,mrnr_no,smrt_trmnl_no,mtinsp_dt,smrt_mtinsp_hour_usgqty
2021-01-01,1970000026,A00,1911WS0059893,2020-11-23 01:00:00,0.000
2021-01-02,1970000026,A00,1911WS0059893,2020-11-23 02:00:00,0.000
2021-01-03,1970000026,A00,1911WS0059893,2020-11-23 03:00:00,0.000
2021-01-04,1970000026,A00,1911WS0059893,2020-11-23 04:00:00,0.000
2021-01-05,1970000026,A00,1911WS0059893,2020-11-23 05:00:00,0.000
...,...,...,...,...,...
2021-06-02,1970000026,A00,1911WS0059893,2020-11-29 20:00:00,0.000
2021-06-03,1970000026,A00,1911WS0059893,2020-11-29 21:00:00,0.000
2021-06-04,1970000026,A00,1911WS0059893,2020-11-29 22:00:00,0.000
2021-06-05,1970000026,A00,1911WS0059893,2020-11-29 23:00:00,0.000


## Grouping 함수

In [36]:
df.smrt_mtinsp_hour_usgqty.mean()

0.005363057324840764

In [45]:
print('mean : ' + str(df['smrt_mtinsp_hour_usgqty'].mean()))
print('max : '  + str(df['smrt_mtinsp_hour_usgqty'].max()))
print('min : '  + str(df['smrt_mtinsp_hour_usgqty'].min()))

mean : 0.005363057324840764
max : 0.078
min : 0.000


In [94]:
dfaa = df.loc[:, ['cstmr_no', 'smrt_mtinsp_hour_usgqty']]
print(dfaa.groupby('cstmr_no').sum())
#print(dfaa.groupby('cstmr_no').mean())
print(dfaa.groupby('cstmr_no').max())
print(dfaa.groupby('cstmr_no').min())
print(dfaa.groupby('cstmr_no').size()) # 묶인갯수

           smrt_mtinsp_hour_usgqty
cstmr_no                          
1970000026                   0.842
           smrt_mtinsp_hour_usgqty
cstmr_no                          
1970000026                   0.078
           smrt_mtinsp_hour_usgqty
cstmr_no                          
1970000026                   0.000
cstmr_no
1970000026    157
dtype: int64


## 날짜 생성하기

In [95]:
pd.date_range('2020-12-01 00:00', periods=10, freq='M')
pd.date_range('2020-12-01 00:00', periods=10, freq='D')
pd.date_range('2020-12-01 00:00', periods=10, freq='H')

DatetimeIndex(['2020-12-01 00:00:00', '2020-12-01 01:00:00',
               '2020-12-01 02:00:00', '2020-12-01 03:00:00',
               '2020-12-01 04:00:00', '2020-12-01 05:00:00',
               '2020-12-01 06:00:00', '2020-12-01 07:00:00',
               '2020-12-01 08:00:00', '2020-12-01 09:00:00'],
              dtype='datetime64[ns]', freq='H')

## 그래프그리기

In [137]:
ts = pd.Series(np.random.randn(1000), index=pd.date_range('1/1/2000', periods=1000))
ts = ts.cumsum() #누적합

In [139]:
type(np.random.randn(1000))

numpy.ndarray

In [106]:
df = pd.DataFrame(np.random.randn(1000, 4), index=ts.index, columns=['A', 'B', 'C', 'D']) #ts집합의 index사용
df = df.cumsum() #누적합
plt.figure(); df.plot(); plt.legend(loc='best')

In [153]:
aaa = pd.DataFrame({'A': 1.,
                    'B': np.array(df2.smrt_mtinsp_hour_usgqty),
                    'C': 'foo'}
                    , index=pd.date_range('20200101', periods=len(df2), freq="M"))

In [154]:
#plt.figure(); aaa.plot(); plt.legend(loc='best')

In [160]:
cust = pd.DataFrame({'age' : np.arange(30,51), 

                    'sales' : np.repeat([65,60,80],7)})
len(cust)

21

In [159]:
freq = np.repeat([3,7,4,2,1,1,2],3)
len(freq) 

21